
# Distributed mongodb

see Sharding 

- https://docs.mongodb.com/v3.0/sharding/


Understand shard keys and how to design queries that would route to only one shard based on the key

Do the example here https://docs.mongodb.com/v3.0/tutorial/deploy-shard-cluster/ launching 1 config server and 3 shards:

### setup

prepare dirs:

    mkdir /data/distributed/cfgsrv1    
    mkdir /data/distributed/shard1
    mkdir /data/distributed/shard2    
    mkdir /data/distributed/shard3

launch configsrv

    mongod --configsvr --dbpath /data/distributed/cfgsrv1  --port 27019
    
launch router

    mongos --configdb localhost:27019

launch 3 mongo servers

    mongod --port 27021 --dbpath /data/distributed/shard1
    mongod --port 27022 --dbpath /data/distributed/shard2
    mongod --port 27023 --dbpath /data/distributed/shard3


use command line to register servers

    mongo --host localhost --port 27017

    sh.addShard( "localhost:27021" )
    sh.addShard( "localhost:27022" )
    sh.addShard( "localhost:27023" )
    
    exit();
    
check shard status

    mongo --host localhost --port 27017
    sh.status()

### sharding databases

create a shared collections

    mongo --host localhost --port 27017

    use mydbs
    sh.enableSharding("mydbs")   
   
    sh.shardCollection("mydbs.casas", { "ciudad": 1  } )
    
    for (i=0; i<30; i++) { db.casas.insert({ "id": 209182, "ciudad": "torino", "superficie": i*10+10}) }   
    for (i=0; i<30; i++) { db.casas.insert({ "id": "20918xx", "ciudad": "madrid", "superficie": i*2+10}) }
    for (i=0; i<10; i++) { db.casas.insert({ "id": 20912, "ciudad": "roma", "superficie": i*2+2}) }   
    
    db.casas.count()
    
    cursor = db.casas.find()
    cursor.explain()
    
connect to each shard and inspect it

    mongo --host localhost --port 27021
    use mydbs
    db.casas.count()
    db.casas.find()
    
    mongo --host localhost --port 27022
    use mydbs
    db.casas.count()
    db.casas.find()
    
    mongo --host localhost --port 27022
    use mydbs
    db.casas.count()
    db.casas.find()